<div style="border:solid green 2px; padding: 20px">
<b>Никита, привет!</b>

Меня зовут Евгений Дащенко, и я буду проверять твой проект. Предлагаю общаться на «ты» 😊 Но если это не удобно - дай знать, и мы перейдем на "вы". 

Моя основная цель — не указать на совершенные тобою ошибки, а поделиться своим опытом и помочь тебе стать аналитиком данных. Ты уже проделал большую работу над проектом, но давай сделаем его еще лучше. Ниже ты найдешь мои комментарии - пожалуйста, не перемещай, не изменяй и не удаляй их. Я буду использовать цветовую разметку:

<div class="alert alert-danger">
<b>Комментарий ревьюера:</b> Так выделены самые важные замечания. Без их отработки проект не будет принят. </div>

<div class="alert alert-warning">
<b>Комментарий ревьюера:</b> Так выделены небольшие замечания. Одна-два таких замечания в проекте допустимы, но если их будет больше - тебе будет необходимо внести исправления. Это как тестовое задание при приеме на работу: очень много мелких ошибок могут стать причиной отказа кандидату. 

</div>

<div class="alert alert-success">
<b>Комментарий ревьюера:</b> Так я выделяю все остальные комментарии.</div>

Давай работать над проектом в диалоге: если ты что-то меняешь в проекте или отвечаешь на мои комменатри — пиши об этом. Мне будет легче отследить изменения, если ты выделишь свои комментарии:
<div class="alert alert-info"> <b>Комментарий студента:</b> Например, вот так.</div>

Всё это поможет выполнить повторную проверку твоего проекта оперативнее. 

<div class="alert alert-info"> <b>Комментарий студента:</b> Привет, рад знакомству, сразу оставлю ссылку на вывод.</div>

[Ссылка на вывод](#Общий_вывод)

# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Здорово, когда есть вступление в сам проект, каждый сможет быстрее разобраться с тем о чем сам проект 👍

</div>

In [1]:
import pandas as pd 
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from pprint import pprint
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
import numpy as np

In [2]:
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

## 1. Откройте и изучите файл

In [3]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [6]:
df['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

Разница классов почти два раза, при разбиении потребуется стратификация

## 2. Разбейте данные на выборки

In [7]:
features = df.drop(columns=['is_ultra'])
target = df['is_ultra']

Разделяем фичи от классов

In [8]:
train_features, test_features, train_target, test_target = train_test_split(
    features, 
    target, 
    test_size=0.2,
    stratify = target,
    random_state = 3481)

Распковываем фичи и классы на тренировочные и тестовые сеты в соотношении 80\20 

In [9]:
train_features, valid_features, train_target, valid_target = train_test_split(
    train_features, 
    train_target, 
    test_size=0.25,
    stratify = train_target,
    random_state = 3481)                                                      

Разделяем тренировочный сет на тренировочный и валидационный по аналогии с первым действием.

### Вывод
В результате разделе мы имеем три сета - Тренировочный\train (60%), Валидационный\valid (20%) и Тестовый\test (20%)

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

    
В целом выборка разделена верно. 
Я бы рекомендовал бы ещё использовать атрибут `stratify` для целевой переменной. Он дает нам безопасность сохранения 20% значений равного нужного нам количества классов в целевой переменной.
    
    
    
</div>

## 3. Исследуйте модели

### Решающее дерево
Используем Randomized Search, что перебрать гиперпараметры модели 

In [10]:
# Максимальная глубина дерева
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
# Минимальное число обектов в каждом листе
min_samples_leaf = [10, 20, 50, 100]
# Минимальное число объектов, необходимое, чтобы разделить узел
min_samples_split = [2, 5, 10]

random_grid = {'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}


In [11]:
dc_model = DecisionTreeClassifier()

In [12]:
rf_random = RandomizedSearchCV(estimator = dc_model, 
                               param_distributions = random_grid, 
                               n_iter = 100, 
                               cv = 3, 
                               verbose=2, 
                               random_state=3481, 
                               n_jobs = -1)

rf_random.fit(train_features, train_target)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] min_samples_split=5, min_samples_leaf=100, max_depth=40 .........
[CV]  min_samples_split=5, min_samples_leaf=100, max_depth=40, total=   0.0s
[CV] min_samples_split=5, min_samples_leaf=100, max_depth=40 .........
[CV]  min_samples_split=5, min_samples_leaf=100, max_depth=40, total=   0.0s
[CV] min_samples_split=5, min_samples_leaf=100, max_depth=40 .........
[CV]  min_samples_split=5, min_samples_leaf=100, max_depth=40, total=   0.0s
[CV] min_samples_split=5, min_samples_leaf=50, max_depth=60 ..........
[CV]  min_samples_split=5, min_samples_leaf=50, max_depth=60, total=   0.0s
[CV] min_samples_split=5, min_samples_leaf=50, max_depth=60 ..........
[CV]  min_samples_split=5, min_samples_leaf=50, max_depth=60, total=   0.0s
[CV] min_samples_split=5, min_samples_leaf=50, max_depth=60 ..........
[CV]  min_samples_split=5, min_samples_leaf=50, max_depth=60, total=   0.0s
[CV] min_samples_split=2, min_samples_leaf=10, max_d

[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV]  min_samples_split=10, min_samples_leaf=10, max_depth=40, total=   0.0s
[CV] min_samples_split=10, min_samples_leaf=10, max_depth=40 .........
[CV]  min_samples_split=10, min_samples_leaf=10, max_depth=40, total=   0.0s
[CV] min_samples_split=10, min_samples_leaf=20, max_depth=40 .........
[CV]  min_samples_split=10, min_samples_leaf=20, max_depth=40, total=   0.0s
[CV] min_samples_split=10, min_samples_leaf=20, max_depth=40 .........
[CV]  min_samples_split=10, min_samples_leaf=20, max_depth=40, total=   0.0s
[CV] min_samples_split=10, min_samples_leaf=20, max_depth=40 .........
[CV]  min_samples_split=10, min_samples_leaf=20, max_depth=40, total=   0.0s
[CV] min_samples_split=5, min_samples_leaf=20, max_depth=40 ..........
[CV]  min_samples_split=5, min_samples_leaf=20, max_depth=40, total=   0.0s
[CV] min_samples_split=5, min_samples_leaf=20, max_depth=40 ..........
[CV]  min_samples_split=5, min_samples_leaf=20, max_depth=40, total=   0.0s
[CV] min_samples_split=5, min_samples

[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:    2.8s finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=DecisionTreeClassifier(class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort=False,
                                                    random_state=None,
                                                    splitter='best')

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Здорово, что исползуем автоматический перебор наших гиперпараметров.
    
---
    
Кстати достаточно большая глубина деревьв в 100 шт, обычно до 10 используют.
    
</div>

In [13]:
rf_random.best_score_

0.7863070539419087

### 3.1 Случайный лес

Используем Randomized Search, что перебрать гиперпараметры модели 

In [14]:
# Количество деревьев в ансамбле
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 100, num = 10)]
# Максимальное число фичей для разделения
max_features = ['auto', 'sqrt']
# Максимальная глубина дерева
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
# Минимальное число объектов, необходимое, чтобы разделить узел
min_samples_split = [2, 5, 10]
# Минимальное число обектов в каждом листе
min_samples_leaf = [10, 20, 50, 100]
# Метод выбора объектов для обучения каждого дерева
bootstrap = [True, False]


random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

    
Очень здорово, что используем дополнительные гиперпараметры в нашей моделе.
    
Что повышает нам вероятность найти лучшую модель.
    
</div>

In [15]:
rfc_model = RandomForestClassifier()

In [16]:
rf_random = RandomizedSearchCV(estimator = rfc_model, 
                               param_distributions = random_grid, 
                               n_iter = 100, 
                               cv = 3, 
                               verbose=2, 
                               random_state=3481, 
                               n_jobs = -1)

rf_random.fit(train_features, train_target)

[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] n_estimators=70, min_samples_split=10, min_samples_leaf=10, max_features=sqrt, max_depth=110, bootstrap=True 
[CV]  n_estimators=70, min_samples_split=10, min_samples_leaf=10, max_features=sqrt, max_depth=110, bootstrap=True, total=   0.3s
[CV] n_estimators=70, min_samples_split=10, min_samples_leaf=10, max_features=sqrt, max_depth=110, bootstrap=True 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s


[CV]  n_estimators=70, min_samples_split=10, min_samples_leaf=10, max_features=sqrt, max_depth=110, bootstrap=True, total=   0.2s
[CV] n_estimators=70, min_samples_split=10, min_samples_leaf=10, max_features=sqrt, max_depth=110, bootstrap=True 
[CV]  n_estimators=70, min_samples_split=10, min_samples_leaf=10, max_features=sqrt, max_depth=110, bootstrap=True, total=   0.2s
[CV] n_estimators=60, min_samples_split=5, min_samples_leaf=100, max_features=auto, max_depth=100, bootstrap=True 
[CV]  n_estimators=60, min_samples_split=5, min_samples_leaf=100, max_features=auto, max_depth=100, bootstrap=True, total=   0.1s
[CV] n_estimators=60, min_samples_split=5, min_samples_leaf=100, max_features=auto, max_depth=100, bootstrap=True 
[CV]  n_estimators=60, min_samples_split=5, min_samples_leaf=100, max_features=auto, max_depth=100, bootstrap=True, total=   0.2s
[CV] n_estimators=60, min_samples_split=5, min_samples_leaf=100, max_features=auto, max_depth=100, bootstrap=True 
[CV]  n_estimators=6

[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  1.1min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators='warn',
                                                    n_jobs=None

In [17]:
rf_random.best_score_

0.8049792531120332

Лучшая оценка, полученная при переборе гипер-параметров модели, из описания к RS понял, что используется кросс-валидация.

###  Логистическая регрессия

In [18]:
lr_model = LogisticRegression()

In [19]:
lr_model.fit(train_features, train_target)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [20]:
lr_prediction = lr_model.predict(valid_features)

In [21]:
accuracy_score(lr_prediction, valid_target)

0.7356143079315708

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

В том числе можно было попробвать и сравнить параметр `C` у логистической регрессии. 
    
---
    
Обрати внимание, что некоторые действия у нас повторяются это говорит о том, что мы можем создать функцию для оптимизации нашего поиска.
    
</div>

## Проверьте модель на тестовой выборке
Проверяем модель Случайного Леса, т.к. она показала лучший скор на валидационной выборке

In [22]:
def evaluate(model, test_features, test_target):
    predictions = model.predict(test_features)
    result = accuracy_score(test_target, predictions)
    return result

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Здорово, что создали функцию для прогнозирования результатов по выбранной модели. 
    
</div>

Функция получает на вход модель, тестовые фичи и тестовые ключи. Возврает score модели

In [23]:
best_grid = rf_random.best_estimator_
accuracy = evaluate(best_grid, test_features, test_target)
print(accuracy)

0.7947122861586314


<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Супер, цель достигнута 👍
    
    
Кстати, перед проверкой нашей модели на тестовой выборке, рекомендую соединять валидационную и обучающую выборки и еще раз обучить модель. Так мы получим бОльшее количество данных для обучения.
    

    
</div>

## 5. (бонус) Проверьте модели на адекватность
Будем использовать DummyClassifier для проверки модели на адекватность. В качестве стратегии выставим uniform, что позволит делать предсказания случайным образом, то есть угадывать.

In [24]:
dummy_model = DummyClassifier(random_state=3481, strategy='uniform')

In [25]:
dummy_model.fit(train_features, train_target) 

DummyClassifier(constant=None, random_state=3481, strategy='uniform')

In [26]:
evaluate(dummy_model,test_features, test_target)

0.4774494556765163

Как видно, скор DummyClassifier существено ниже скора выбранной модели, можно считать, что выбранная модель адекватна.

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Согласен.
    
В том числе, чтобы посчитать константную модель мы также могли посчитать доли наших классов.
    
</div>

<div class="alert alert-danger">
<b>Комментарий ревьюера:</b>

<s>Финальный вывод и есть главный результат твоей работы. Стоит писать его подробно по результатам проведенной работы. В нем можно приводить полученные в ходе работы значения. Также можно расписать все, что было сделано в работе.
    
Самое главное — это вывод должен содержать в себе ответ на поставленную задачу.

</div>


<div class="alert alert-info"> <b>Комментарий студента:</b> Общий вывод есть, ниже, после чек-листа.)</div>


<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

    
Достаточно детально проделанная работа и это не может не радовать 😊
    
Давай поправим мои комментарии и будем двигаться дальше.
    
Если у тебя вдруг есть какие–либо вопросы, то я с радостью отвечу на них 😊
       

</div>

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75


## Общий вывод
В результате работы были изучены задача и представленные данные, выполнено стратифицированное разделение на выборки. 
На полученных данных были протестированы три ml-модели (решающее дерево, ансамбль решающих деревьев, логистическая регрессия)
Самую высокую точность на валидационной выборке (0.808) показал случайный лес. Результат лучшей модели при проверки на тестовой выборке составил - 0.786
При помощи DummyClassifier использованного как имитирование угадывания правильных ответов была доказана адекватность модели.
В результате мы получили модель, способную предсказывать тариф по входным параметрам c заданым уровнем точности.

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>
    
Сори 🤦‍♂️
    
---

Выводы понятны и логичны, а самое главное подкреплены выявленными фактами.
       
В целом сам проект выполнен на хорошем уровне! На протяжении всей работы чувствуется глубина проработки задачи и это безусловный плюс для нас. Было использовано большое количество методов, которые будут помогать тебе в дальнейших проектах.
        
Поздравляю с успешным завершением проекта 😊👍
    
И желаю успехов в новых работах 😊
    

    
</div>